In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园 - JAX F-VLM

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_jax_fvlm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_jax_fvlm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>                                                                                               <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_jax_fvlm.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI Workbench中打开
    </a>
  </td>
</table>

**_注意_**：此笔记本已在以下环境中进行测试：

- Python 版本 = 3.9

## 概述

本笔记本演示了训练和部署[JAX F-VLM模型](https://github.com/google-research/google-research/tree/master/fvlm)用于[开放词汇目标检测和实例分割](https://arxiv.org/abs/2209.15639)任务，并在Vertex AI上用于在线预测。

### 目标

在本教程中，您将学习如何：

- 准备训练数据集。
- 训练一个新的JAX F-VLM模型。
- 将模型上传到[模型注册表](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)。
- 在[端点](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints)部署模型。
- 运行用于开放词汇图像对象检测和实例分割的在线预测。

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI模型注册表
- Vertex AI在线预测

数据集

这份笔记本使用以下预测图像作为示例:

图像: https://pixabay.com/nl/photos/het-fruit-eten-citroen-limoen-3134631/

知识共享许可证: https://pixabay.com/nl/service/terms/

您也可以使用自己的自定义预测图像，方法是修改本笔记本中`DEMO_IMAGE_PATH`变量。

费用

本教程使用Google Cloud的付费组件：
* Vertex AI
* Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您预计的使用情况生成费用估算。

## 安装

安装以下所需的包以执行此笔记本。

In [ ]:
# Install the packages.
! pip3 install --upgrade google-cloud-aiplatform
! sudo apt-get install -y subversion protobuf-compiler python3-lxml\
  python3-pip python3-dev git unzip
# Note: The following libraries are pinned down versions of:
# https://github.com/google-research/google-research/blob/master/fvlm/requirements.txt
! pip install tensorflow==2.12.0
! pip install tensorflow-datasets==4.9.2
! pip install numpy==1.23.5
! pip install torch==2.0.1+cu118 --extra-index-url https://download.pytorch.org/whl/cu118
! pip install torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118
! pip install opencv-python==4.7.0.72
! pip install tqdm==4.65.0
! pip install git+https://github.com/openai/CLIP.git@a1d071733d7111c9c014f024669f959182114e33
! pip install Pillow==9.1.1
! pip install orbax-checkpoint==0.3.3
! pip install gin-config==0.5.0
! pip install pycocotools==2.0.6
! pip install contextlib2==21.6.0
! pip install ml-collections==0.1.1
! pip install chex==0.1.7
! pip install optax==0.1.5
# Dependencies already included. Use no-deps to not update numpy.
! pip install --no-deps flax==0.7.2
! pip install --no-deps clu==0.0.9
! pip install jax[cuda11_cudnn86]==0.4.9 \
  --find-links https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
! pip install Cython
! pip install git+https://github.com/cocodataset/cocoapi#subdirectory=PythonAPI

# Get F-VLM repository by using svn to avoid downloading entire google-research repository.
%cd
! rm -rf ./fvlm
! svn export -r 60422 https://github.com/google-research/google-research/trunk/fvlm

将F-VLM检查点下载到“fvlm/checkpoints”文件夹中。

In [ ]:
%cd fvlm/checkpoints
! ./download.sh
%cd ../../

下载 COCO嵌入并设置数据准备代码。

In [ ]:
%cd fvlm
! bash ./scripts/download_precomputed_embeddings.sh
! git clone https://github.com/tensorflow/tpu.git
! git clone http://github.com/tensorflow/models tf-models
%cd tf-models/research
! protoc object_detection/protos/*.proto --python_out=.
%cd ../../../

重新启动内核

In [ ]:
# Restart kernel after installs so that your environment can access the new packages.
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 开始之前

### 设置你的Google Cloud项目

**无论您使用什么笔记本环境，以下步骤都是必须的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用额度用于您的计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 检查您的加速器[配额](https://console.cloud.google.com/iam-admin/quotas)。此笔记本使用TPU V3 8核心，您可以在您的地区为`Vertex AI API`服务的8个核心申请`每个地区的自定义模型训练TPU V3核心`配额。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下步骤：
- 运行 `gcloud config list`。
- 运行 `gcloud projects list`。
- 查看支持页面：[定位项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动认证。请按照以下相关说明操作。

1. Vertex AI工作台
* 无需操作，因为您已经通过身份验证。

2. 本地JupyterLab实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 查看如何将云存储权限授予您的服务账户，请访问https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

导入库

In [ ]:
import base64
import datetime
import functools
import json
import os
import sys
from io import BytesIO

import gin
import jax
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tqdm
from PIL import Image

# Work in fvlm directory in order to create gin config later.
os.chdir(os.path.join(os.path.expanduser("~"), "fvlm"))
sys.path.append(os.getcwd())
from demo_utils import input_utils as inputs
from demo_utils import vis_utils
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from utils import clip_utils

### 初始化用于 Python 的 Vertex AI SDK

为您的项目初始化用于 Python 的 Vertex AI SDK。

In [ ]:
staging_bucket = os.path.join(BUCKET_URI, "jax_fvlm_staging")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=staging_bucket)

### 定义常量

In [ ]:
REGION_PREFIX = REGION.split("-")[0]

# The F-VLM model to use. Choose between 'resnet_50', 'resnet_50x4', or 'resnet_50x16'.
MODEL = "resnet_50"

# COCO dataset constants. Only used for training with COCO dataset.
BASE_ANNOTATION_URL = "http://images.cocodataset.org/annotations"
INSTANCES_FILE = "annotations_trainval2017.zip"
IMAGE_INFO_FILE = "image_info_test2017.zip"
BASE_IMAGE_URL = "http://images.cocodataset.org/zips"
TRAIN_IMAGE_FILE = "train2017.zip"
VAL_IMAGE_FILE = "val2017.zip"
TRAIN_ANNOTATION_PATH = "./annotations/instances_train2017.json"
VAL_ANNOTATION_PATH = "./annotations/instances_val2017.json"
LOCAL_CATEGORY_EMBEDDING_PATH = f"./embeddings/{MODEL}/coco_embed.npy"
# Paths to save annotations for training example.
TRAIN_TOY_ANNOTATION_PATH = "./annotations/toy_instances_train2017.json"
VAL_TOY_ANNOTATION_PATH = "./annotations/toy_instances_val2017.json"
TRAIN_IMAGE_DIR = "./train2017"
VAL_IMAGE_DIR = "./val2017"
TRAIN_DATA_PREFIX = "train"
VAL_DATA_PREFIX = "val"
# Local directory to save tfrecord dataset.
LOCAL_TFRECORD_DIR = "./tfrecord"
# GCS directory to upload tfrecord dataset.
GCS_TFRECORD_PATH = f"{BUCKET_URI}/dataset"

# GCS location of the category embeddings file.
GCS_CATEGORY_EMBEDDING_PATH = f"{BUCKET_URI}/embeddings/{MODEL}/embed.npy"
# An upper bound on the number of classes.
MAX_NUM_CLS = 91
# URL to download checkpoint.
CHECKPOINT_URL = (
    "https://storage.googleapis.com/cloud-tpu-checkpoints/detection/projects/fvlm/"
    f"jax_checkpoints/{MODEL.replace('resnet_', 'r')}_checkpoint_184000"
)

# The local path to the F-VLM folder.
F_VLM_FOLDER = "."  # Current directory.
# Train config template path.
LOCAL_TRAIN_CONFIG_TEMPLATE_PATH = f"{F_VLM_FOLDER}/configs/fvlm_train_and_eval.gin"
# Final train config path.
GCS_TRAIN_CONFIG_PATH = f"{BUCKET_URI}/fvlm_train_and_eval.gin"
# Base output path for training artifacts.
GCS_TRAIN_BASE_PATH = f"{BUCKET_URI}/train"
# Final train config path.
GCS_TRAIN_CONFIG_PATH = f"{GCS_TRAIN_BASE_PATH}/fvlm_train_and_eval.gin"
# Training container image.
TRAIN_DOCKER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/jax-f-vlm-train"
GCS_TRAIN_DATA_PATH = f"{GCS_TFRECORD_PATH}/{TRAIN_DATA_PREFIX}*.tfrecord"
GCS_VAL_DATA_PATH = f"{GCS_TFRECORD_PATH}/{VAL_DATA_PREFIX}*.tfrecord"
GCS_VAL_ANNOTATION_FILE = f"{GCS_TRAIN_BASE_PATH}/val_annotation.json"
LOCAL_CHECKPOINT_FILE = (
    f"./checkpoints/{MODEL.replace('resnet_', 'r')}_checkpoint_184000"
)
# GCS location to upload pretrained checkpoint to.
GCS_CHECKPOINT_DIR = f"{GCS_TRAIN_BASE_PATH}/pretrained_checkpoint"

# The train/eval batch sizes should be divisible by the number of GPUs used.
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
# Whether to predict mask.
# Set False for object detection task and True for segmentation task.
INCLUDE_MASK = True
# Number of TPU cores to be used.
NUM_CORES = 8
# Total train steps.
TRAIN_STEPS = 1000  # 1 epoch is 3787 steps in this example.
# Total evaluation steps.
EVAL_STEPS = 156
# Initial learning rate.
INIT_LEARNING_RATE = 0.001
# Training image size.
OUTPUT_SIZE = 1024
# Variable dtype. bfloat16 or float32 are supported.
DTYPE = "bfloat16"

# The pre-built TF SavedModel conversion docker image.
MODEL_CONVERSION_DOCKER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/jax-f-vlm-model-conversion"

# The pre-built prediction docker image.
OPTIMIZED_TF_RUNTIME_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-cpu.nightly:latest"
)
# The max number of boxes to draw on the output image.
MAX_BOXES_TO_DRAW = 25
# The minimum score required to draw a detected object.
MIN_SCORE_THRESH = 0.2  # @param {type:"slider", min:0, max:0.9, step:0.05}
# The local path to the output image.
OUTPUT_IMAGE_PATH = "./output.jpg"
# The F-VLM SavedModel folder which takes image and text embeddings as inputs.
GCS_SAVED_MODEL_DIR = f"{BUCKET_URI}/saved_model"
# The converted SavedModel folder which takes jpeg bytes and text-embeddings bytes as inputs.
LOCAL_CONVERTED_SAVED_MODEL_DIR = f"{F_VLM_FOLDER}/converted_saved_model"
# The Cloud Storage location for the converted SavedModel.
GCS_CONVERTED_SAVED_MODEL_DIR = f"{BUCKET_URI}/fvlm_saved_model"

### 定义常用函数

本节定义以下函数：

- 将输入图像加载并转换为所需的预测格式。
- 可视化检测输出。
- 获取GCS Fuse路径。
- 获取带有当前时间的作业名称。
- 保存 COCO 注释文件的子集。

In [ ]:
def convert_numpy_array_to_byte_string_via_tf_tensor(np_array):
    """Serializes a numpy array to tensor bytes."""
    tensor_array = tf.convert_to_tensor(np_array)
    tensor_byte_string = tf.io.serialize_tensor(tensor_array)
    return tensor_byte_string.numpy()


def generate_text_embeddings(categories):
    """Generates text embeddings in numpy format from object categories."""
    clip_text_fn = clip_utils.get_clip_text_fn(MODEL)
    class_clip_features = []
    print("Computing custom category text embeddings.")
    for cls_name in tqdm.tqdm(categories, total=len(categories)):
        cls_feat = clip_text_fn(cls_name)
        class_clip_features.append(cls_feat)
    text_embeddings = np.concatenate(class_clip_features, axis=0)
    embed_path = (
        f'{F_VLM_FOLDER}/data/{MODEL.replace("resnet_", "r")}_bg_empty_embed.npy'
    )
    background_embedding, empty_embeddings = np.load(embed_path)
    background_embedding = background_embedding[np.newaxis, Ellipsis]
    empty_embeddings = empty_embeddings[np.newaxis, Ellipsis]
    tile_empty_embeddings = np.tile(
        empty_embeddings, (MAX_NUM_CLS - len(categories) - 1, 1)
    )
    # Concatenate 'background' and 'empty' embeddings.
    text_embeddings = np.concatenate(
        (background_embedding, text_embeddings, tile_empty_embeddings), axis=0
    )
    return text_embeddings


def get_jpeg_bytes(local_image_path, new_width=-1):
    """Returns jpeg bytes given an image path and resizes if required."""
    image = Image.open(local_image_path)
    if new_width <= 0:
        new_image = image
    else:
        width, height = image.size
        print("original input image size: ", width, " , ", height)
        new_height = int(height * new_width / width)
        print("new input image size: ", new_width, " , ", new_height)
        new_image = image.resize((new_width, new_height))
    buffered = BytesIO()
    new_image.save(buffered, format="JPEG")
    return buffered.getvalue()


def generate_prediction_output_image(
    input_image_path, prediction_output, output_image_path, categories
):
    """Generates prediction output image with detected objects and bounding boxes."""
    # Generate tensors from prediction outputs.
    prediction_output_tensor = {}
    for key, val in prediction_output.items():
        prediction_output_tensor[key] = tf.expand_dims(
            tf.convert_to_tensor(val), axis=0
        )
    prediction_output_tensor["num_detections"] = tf.cast(
        prediction_output_tensor["num_detections"], tf.int32
    )
    # Generate image embeddings for the input image.
    with open(input_image_path, "rb") as f:
        np_image = np.array(Image.open(f))
    parser_fn = inputs.get_maskrcnn_parser()
    data = parser_fn({"image": np_image, "source_id": np.array([0])})
    np_data = jax.tree_map(lambda x: x.numpy()[np.newaxis, Ellipsis], data)
    image_embeddings = np_data.pop("images")
    labels = np_data.pop("labels")
    # Generate visualization.
    print("Preparing visualization.")
    id_mapping = {(i + 1): c for i, c in enumerate(categories)}
    id_mapping[0] = "background"
    for k in range(len(categories) + 2, MAX_NUM_CLS):
        id_mapping[k] = "empty"
    category_index = inputs.get_category_index(id_mapping)
    maskrcnn_visualizer_fn = functools.partial(
        vis_utils.visualize_boxes_and_labels_on_image_array,
        category_index=category_index,
        use_normalized_coordinates=False,
        max_boxes_to_draw=MAX_BOXES_TO_DRAW,
        min_score_thresh=MIN_SCORE_THRESH,
        skip_labels=False,
    )
    vis_image = vis_utils.visualize_instance_segmentations(
        prediction_output_tensor,
        image_embeddings,
        labels["image_info"],
        maskrcnn_visualizer_fn,
    )
    pil_vis_image = Image.fromarray(vis_image, mode="RGB")
    pil_vis_image.save(output_image_path)
    print("Completed saving the output image at: ", output_image_path)


def gcs_fuse_path(path: str) -> str:
    """Try to convert path to gcsfuse path if it starts with gs:// else do not modify it."""
    path = path.strip()
    if path.startswith("gs://"):
        return "/gcs/" + path[5:]
    return path


def get_job_name_with_datetime(prefix: str):
    """Gets a job name by adding current time to prefix."""
    return prefix + datetime.datetime.now().strftime("_%Y%m%d_%H%M%S")


def save_subset_annotation(input_annotation_path, output_annotation_path):
    """Saves a subset of COCO annotation json file with CCA 4.0 license."""

    with open(input_annotation_path) as f:
        coco_json = json.load(f)

    img_ids = set()
    images = []
    annotations = []

    for img in coco_json["images"]:
        if img["license"] in [4, 5]:  # CCA 4.0 license.
            img_ids.add(img["id"])
            images.append(img)

    for ann in coco_json["annotations"]:
        if ann["image_id"] in img_ids:
            annotations.append(ann)

    new_json = {
        "info": coco_json["info"],
        "licenses": coco_json["licenses"],
        "images": images,
        "annotations": annotations,
        "categories": coco_json["categories"],
    }

    with open(output_annotation_path, "w") as f:
        json.dump(new_json, f)

## 训练新模型

本部分显示如何训练新模型：

1. 将输入数据转换为tfrecord格式。
2. 上传tfrecord文件和类别嵌入到GCS位置。
3. 创建自定义训练作业以训练新的JAX模型。

如果您想使用提供的预训练保存的模型，您可以跳过本部分，直接转到“将F-VLM SavedModel转换为支持更小输入尺寸”的部分。

### 准备用于训练的输入数据

将F-VLM SavedModels和检查点下载到`fvlm/checkpoints`文件夹中。

In [ ]:
%cd checkpoints
! ./download.sh
! wget {CHECKPOINT_URL}
%cd ../../

下载并解压COCO标注和图像。这可能需要大约25分钟。只有当你在训练时使用COCO数据集时才需要这一部分。如果你想使用自己的数据集，那么跳过这一部分，并通过将你的训练/验证图像分别保存在`TRAIN_IMAGE_DIR`和`VAL_IMAGE_DIR`中，并分别在`TRAIN_TOY_ANNOTATION_PATH`和`VAL_TOY_ANNOTATION_PATH`中编写训练/验证COCO格式的标注。

In [ ]:
# Download and unzip instances file.
! wget -nd -c {BASE_ANNOTATION_URL}/{INSTANCES_FILE}
! unzip -nq {INSTANCES_FILE}

# Download and unzip image info file.
! wget -nd -c {BASE_ANNOTATION_URL}/{IMAGE_INFO_FILE}
! unzip -nq {IMAGE_INFO_FILE}

# Download and unzip train images.
! wget -nd -c {BASE_IMAGE_URL}/{TRAIN_IMAGE_FILE}
! unzip -nq {TRAIN_IMAGE_FILE}

# Download and unzip eval images.
! wget -nd -c {BASE_IMAGE_URL}/{VAL_IMAGE_FILE}
! unzip -nq {VAL_IMAGE_FILE}

# Remove zip files.
! rm {INSTANCES_FILE}
! rm {IMAGE_INFO_FILE}
! rm {TRAIN_IMAGE_FILE}
! rm {VAL_IMAGE_FILE}

使用COCO注释的子集创建玩具注释文件。仅当您在训练时使用COCO数据集时才需要此部分。如果要使用自己的数据集，则跳过此部分。

In [ ]:
save_subset_annotation(
    input_annotation_path=TRAIN_ANNOTATION_PATH,
    output_annotation_path=TRAIN_TOY_ANNOTATION_PATH,
)
save_subset_annotation(
    input_annotation_path=VAL_ANNOTATION_PATH,
    output_annotation_path=VAL_TOY_ANNOTATION_PATH,
)

将数据集保存为TF Record。这可能需要大约10分钟。

In [ ]:
! PYTHONPATH="tf-models:tf-models/research" python3 ./tpu/tools/datasets/create_coco_tf_record.py \
--logtostderr \
--include_masks \
--image_dir={TRAIN_IMAGE_DIR} \
--object_annotations_file={TRAIN_TOY_ANNOTATION_PATH} \
--output_file_prefix={LOCAL_TFRECORD_DIR}/{TRAIN_DATA_PREFIX} \
--num_shards=256

In [ ]:
! PYTHONPATH="tf-models:tf-models/research" python3 ./tpu/tools/datasets/create_coco_tf_record.py \
--include_masks \
--image_dir={VAL_IMAGE_DIR} \
--object_annotations_file={VAL_TOY_ANNOTATION_PATH} \
--output_file_prefix={LOCAL_TFRECORD_DIR}/{VAL_DATA_PREFIX} \
--num_shards=256

### 将tfrecord文件、预训练检查点和类别嵌入上传到GCS位置

In [ ]:
! gsutil -m rm -R -f {GCS_TFRECORD_PATH}
! gsutil -m cp -R {LOCAL_TFRECORD_DIR} {GCS_TFRECORD_PATH}
! gsutil -m cp {VAL_TOY_ANNOTATION_PATH} {GCS_VAL_ANNOTATION_FILE}
! gsutil -m cp {LOCAL_CHECKPOINT_FILE} {GCS_CHECKPOINT_DIR}/checkpoint_0

上传类别嵌入。如果你想使用自己的数据集，那么跳过这个单元格。

In [ ]:
! gsutil -m cp {LOCAL_CATEGORY_EMBEDDING_PATH} {GCS_CATEGORY_EMBEDDING_PATH}

如果你想使用自己的数据集，取消下面的单元格注释以创建你自己的类别嵌入。

In [ ]:
# CATEGORIES = [
#     "kiwi",
#     "orange",
#     "lemon",
#     "blackberry",
#     "pine cone",
#     "red orange",
#     "table",
#     "spoon",
#     "pine needles",
#     "seed",
# ]
# text_embeddings = generate_text_embeddings(categories=CATEGORIES)
# with tf.io.gfile.GFile(GCS_CATEGORY_EMBEDDING_PATH, "w") as f:
#     np.save(f, text_embeddings)

### 创建自定义训练作业并训练一个新模型

设置训练超参数。要查看所有可自定义超参数的完整列表，请查看[config template file](https://github.com/google-research/google-research/blob/e1f9fae637db06ba885217518cbbf7f4fa4b9d7b/fvlm/configs/fvlm_train_and_eval.gin)。

In [ ]:
assert TRAIN_BATCH_SIZE % NUM_CORES == 0, "NUM_CORES must divide TRAIN_BATCH_SIZE."
assert EVAL_BATCH_SIZE % NUM_CORES == 0, "NUM_CORES must divide EVAL_BATCH_SIZE."
config_overrides = [
    "evaluate.host_evaluator = None",  # For evaluation without COCO annotation json file.
    "evaluate.eval_metrics = {'coco_metric': @COCODetectionMetric}",  # For evaluation without COCO annotation json file.
    f"TRAIN_FILE_PATTERN = '{gcs_fuse_path(GCS_TRAIN_DATA_PATH)}'",
    f"EVAL_FILE_PATTERN = '{gcs_fuse_path(GCS_VAL_DATA_PATH)}'",
    f"TRAIN_BS = {TRAIN_BATCH_SIZE}",
    f"EVAL_BS = {EVAL_BATCH_SIZE}",
    f"TRAIN_STEPS = {TRAIN_STEPS}",
    f"EVAL_STEPS = {EVAL_STEPS}",
    f"EMBED_PATH = '{GCS_CATEGORY_EMBEDDING_PATH}'",
    f"CATG_PAD_SIZE = {MAX_NUM_CLS}",
    f"INCLUDE_MASK = {INCLUDE_MASK}",
    f"step_learning_rate_with_linear_warmup.init_learning_rate = {INIT_LEARNING_RATE}",
    f"OUTPUT_SIZE = {OUTPUT_SIZE}",
    f"get_host_evaluator.annotation_file = '{GCS_VAL_ANNOTATION_FILE}'",
    f"train.pretrain_dir = '{GCS_CHECKPOINT_DIR}'",
    f"DTYPE = %jnp.{DTYPE}",
]
print(config_overrides)

gin.parse_config_files_and_bindings(
    [LOCAL_TRAIN_CONFIG_TEMPLATE_PATH], config_overrides, finalize_config=False
)
config = gin.config_str()

with tf.io.gfile.GFile(GCS_TRAIN_CONFIG_PATH, "w") as f:
    f.write(config)

使用Vertex AI SDK 创建并运行训练作业，使用模型工具 JAX F-VLM 训练 Docker。

In [ ]:
JOB_NAME = get_job_name_with_datetime(prefix="jax_fvlm")

docker_args_list = [
    f"--output_dir={gcs_fuse_path(GCS_TRAIN_BASE_PATH)}",
    f"--config_path={gcs_fuse_path(GCS_TRAIN_CONFIG_PATH)}",
    "--mode=train_and_eval",
]
print(docker_args_list)

In [ ]:
# Click on the generated link in the output under "View backing custom job:" to see your run in the Cloud Console.
# The job will run for appoximately 5 minutes in the current settings.
job = aiplatform.CustomContainerTrainingJob(
    display_name=JOB_NAME,
    container_uri=TRAIN_DOCKER_URI,
)
model = job.run(
    args=docker_args_list,
    base_output_dir=f"{GCS_TRAIN_BASE_PATH}",
    replica_count=1,
    machine_type="cloud-tpu",
    accelerator_type="TPU_V3",
    accelerator_count=NUM_CORES,
)

将之前细调的JAX F-VLM模型转换为TF SavedModel，用于在线预测。

In [ ]:
JOB_NAME = get_job_name_with_datetime(prefix="jax_model_conversion")

docker_args_list = [
    f"--input_dir={gcs_fuse_path(GCS_TRAIN_BASE_PATH)}",
    f"--output_dir={gcs_fuse_path(GCS_SAVED_MODEL_DIR)}",
    f"--max_num_classes={MAX_NUM_CLS}",
    f"--model_name={MODEL}",
    f"--include_mask={INCLUDE_MASK}",
]
print(docker_args_list)

In [ ]:
# Create and run the model conversion job.
# Click on the generated link in the output under "View backing custom job:" to see your run in the Cloud Console.
container_uri = MODEL_CONVERSION_DOCKER_URI
job = aiplatform.CustomContainerTrainingJob(
    display_name=JOB_NAME,
    container_uri=container_uri,
)
model_conversion_workdir = os.path.join(BUCKET_URI, JOB_NAME)
model = job.run(
    args=docker_args_list,
    base_output_dir=f"{model_conversion_workdir}",
    replica_count=1,
    machine_type="n1-highmem-8",
)

将F-VLM SavedModel转换为支持较小输入大小

F-VLM SavedModel接受图像嵌入和文本嵌入作为输入。但是，您不能直接将这些输入发送到Vertex AI Online Prediction，因为预测请求大小有1.5 MB的限制。因此，您将首先将SavedModel格式转换为接受jpeg字节和文本嵌入字节作为输入的格式。这种修改后的输入格式将满足1.5 MB的限制要求。

如果您没有训练新模型，请取消注释并运行下面的单元格以使用提供的保存的模型。

In [ ]:
# # Path to the downloaded pre-trained saved model.
# GCS_SAVED_MODEL_DIR = f'{F_VLM_FOLDER}/checkpoints/{MODEL.replace("resnet_","r")}'

In [ ]:
def preprocess_jpeg_byte_string(tensor_byte_string):
    """Converts jpeg bytes to image embeddings as an input for the original F-VLM SavedModel."""
    decoded_image_tensor = tf.io.decode_jpeg(tensor_byte_string, channels=3)
    parser_fn = inputs.get_maskrcnn_parser()
    parser_output = parser_fn({"image": decoded_image_tensor})
    image_embeddings_tensor = parser_output["images"]
    return image_embeddings_tensor


def preprocess_text_embeddings_byte_string(tensor_byte_string):
    """Converts text-embeddings bytes to text-embeddings as an input for the original F-VLM SavedModel."""
    return tf.io.parse_tensor(tensor_byte_string, tf.float32)


def get_serve_fn(model):
    """Creates a serving function for the modified SavedModel which takes jpeg bytes and text-embeddings bytes as an input."""

    @tf.function(
        input_signature=[
            tf.TensorSpec([None], tf.string),
            tf.TensorSpec([None], tf.string),
        ]
    )
    def serve_fn(image_jpeg_bytes_inputs, text_embeddings_bytes_inputs):
        image_embeddings_tensor = tf.map_fn(
            preprocess_jpeg_byte_string, image_jpeg_bytes_inputs, dtype=tf.bfloat16
        )
        text_embeddings_tensor = tf.map_fn(
            preprocess_text_embeddings_byte_string,
            text_embeddings_bytes_inputs,
            dtype=tf.float32,
        )
        return model({"image": image_embeddings_tensor, "text": text_embeddings_tensor})

    return serve_fn


! rm -rf {LOCAL_CONVERTED_SAVED_MODEL_DIR}
model = tf.saved_model.load(GCS_SAVED_MODEL_DIR)
signatures = {
    "serving_default": get_serve_fn(model=model).get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string), tf.TensorSpec([None], tf.string)
    )
}
tf.saved_model.save(model, LOCAL_CONVERTED_SAVED_MODEL_DIR, signatures=signatures)
print("Saved the converted SavedModel to directory: ", LOCAL_CONVERTED_SAVED_MODEL_DIR)

将本地转换后的TF SavedModel 复制到云存储。

In [ ]:
! gsutil -m rm -R -f {GCS_CONVERTED_SAVED_MODEL_DIR}
! gsutil -m cp -R {LOCAL_CONVERTED_SAVED_MODEL_DIR} {GCS_CONVERTED_SAVED_MODEL_DIR}
! gsutil ls {GCS_CONVERTED_SAVED_MODEL_DIR}

## 执行在线预测
使用转换后的TF SavedModel 运行在线预测。

上传TF SavedModel，并部署到一个端点用于预测。这个步骤可能需要最多15分钟来完成。

In [ ]:
serving_env = {
    "MODEL_ID": "F-VLM-JAX-",
    "DEPLOY_SOURCE": "notebook",
}

jax_fvlm_model = aiplatform.Model.upload(
    display_name="jax_fvlm",
    artifact_uri=GCS_CONVERTED_SAVED_MODEL_DIR,
    serving_container_image_uri=OPTIMIZED_TF_RUNTIME_IMAGE_URI,
    serving_container_args=[],
    location=REGION,
    serving_container_environment_variables=serving_env,
)

jax_fvlm_endpoint = jax_fvlm_model.deploy(
    deployed_model_display_name="jax_fvlm_deployed",
    traffic_split={"0": 100},
    machine_type="n1-highmem-16",
    min_replica_count=1,
    max_replica_count=1,
)

准备输入预测图像。

注意：您可以根据需要修改输入图像。

In [ ]:
# Local path to the prediction image.
DEMO_IMAGE_PATH = "./prediction_image.jpg"
# Download the prediction image.
! wget -O {DEMO_IMAGE_PATH} https://cdn.pixabay.com/photo/2018/02/06/12/37/fruit-3134631_1280.jpg

在下面的单元格中定义您自己的类别。

In [ ]:
CATEGORIES = [
    "kiwi",
    "orange",
    "lemon",
    "blackberry",
    "pine cone",
    "red orange",
    "table",
    "spoon",
    "pine needles",
    "seed",
]

准备jpeg字节和文本嵌入字节输入。

In [ ]:
image_jpeg_bytes_inputs = get_jpeg_bytes(
    local_image_path=DEMO_IMAGE_PATH, new_width=1024
)

text_embeddings = generate_text_embeddings(categories=CATEGORIES)
text_embeddings_bytes_inputs = convert_numpy_array_to_byte_string_via_tf_tensor(
    text_embeddings
)

使用base-64编码后跟UTF-8解码来封装字节输入，然后将它们发送到用于预测的端点。Vertex AI预测服务将根据`b64`关键字自动将这些输入字符串转换回字节。

**注意：由于模型的一次性JIT编译，第一次预测可能需要最多2分钟。这可能会导致下面的超时错误。如果您收到超时错误，请等待2分钟后再次运行预测。之后不会再出现超时错误。**
后续的预测需要4秒钟才能完成。

In [ ]:
instances_list = [
    {
        "image_jpeg_bytes_inputs": {
            "b64": base64.b64encode(image_jpeg_bytes_inputs).decode("utf-8")
        },
        "text_embeddings_bytes_inputs": {
            "b64": base64.b64encode(text_embeddings_bytes_inputs).decode("utf-8")
        },
    }
]
instances = [json_format.ParseDict(s, Value()) for s in instances_list]
prediction_output = jax_fvlm_endpoint.predict(instances=instances).predictions[0]

生成带有预测边界框、标签和概率的输出图像。输出图像将被保存到 `OUTPUT_IMAGE_PATH`。

In [ ]:
generate_prediction_output_image(
    input_image_path=DEMO_IMAGE_PATH,
    prediction_output=prediction_output,
    output_image_path=OUTPUT_IMAGE_PATH,
    categories=CATEGORIES,
)

img = Image.open(OUTPUT_IMAGE_PATH)
plt.imshow(img)

清理

要清理此项目中使用的所有 Google Cloud 资源，您可以[删除用于本教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Delete endpoint resource.
jax_fvlm_endpoint.delete(force=True)

# Delete model resource.
jax_fvlm_model.delete()

# Delete Cloud Storage objects that were created.
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI